In [2]:
from __future__ import print_function
import pandas as pd
import matplotlib.pyplot as plt

from lime.lime_tabular import RecurrentTabularExplainer
from tqdm import tqdm

from sp_modif.model_function import *
from sp_modif.methods import *
from sp_modif.data_prep import *
from sp_modif.evaluator import *
from sp_modif.SHAP import *
from sp_modif.L2X import *

In [2]:
tf.__version__

'2.15.0'

In [3]:
# Function to generate explanations
# Function explanation for lime

def get_lime_explanation(data, e, L2X=False) :
    # e  = fn = model.predict
    # Iniatialisation
    df_expplanation = pd.DataFrame(columns=[str(i) for i in range(data.shape[1]*data.shape[2])])

    # Get explanations
    for row in range(data.shape[0]) : 
        explanation = lime_explainer.explain_instance(data[row],
                                                      e,
                                                      num_features=data.shape[1]*data.shape[2]) 
        # fn = model.predict, initialize lime_explainer = Reccurent()
        lime_values = explanation.local_exp[1]
        # Add explanation in df_explanation
        lime_dict = {}
        for tup in lime_values :
            lime_dict[str(tup[0])] = tup[1]
        df_expplanation.loc[len(df_expplanation)] = lime_dict
    
    return df_expplanation

# # Function explanation for others
def get_explainations(data, e, L2X = False) :
    
    # df diemnsion
    if L2X==True :
        X_to_def_col = data[0:1]
        explanation_test = e.explain(X_to_def_col.reshape((X_to_def_col.shape[0], -1)))
        num_columns = explanation_test.flatten().shape[0]
        
    else : 
        explanation_test = e.explain(data[0:1])
        num_columns = explanation_test.flatten().shape[0]
    
    # Iniatialisation
    df_expplanation = pd.DataFrame(columns=[str(i) for i in range(num_columns)])

    # Get explanations
    for row in range(data.shape[0]) :
        if L2X==True:
            X_row = data[row:row+1]
            explanation = e.explain(X_row.reshape((X_row.shape[0], -1)))
        else :
            explanation = e.explain(data[row:row+1])
        # Add explanation in df_explanation
        explanation = explanation.flatten()
        feature_dict = {}
        for i in range(num_columns) :
            feature_dict[str(i)] = explanation[i]
        df_expplanation.loc[len(df_expplanation)] = feature_dict
    
    return df_expplanation

In [3]:
import inspect
print(inspect.getsource(prep_data)) # See a structure of a function

def prep_data(train, test, drop_sensors, remaining_sensors, alpha):
    X_train_interim = add_operating_condition(train.drop(drop_sensors, axis=1))
    X_test_interim = add_operating_condition(test.drop(drop_sensors, axis=1))

    X_train_interim, X_test_interim = condition_scaler(X_train_interim, X_test_interim, remaining_sensors)

    X_train_interim = exponential_smoothing(X_train_interim, remaining_sensors, 0, alpha)
    X_test_interim = exponential_smoothing(X_test_interim, remaining_sensors, 0, alpha)
    
    return X_train_interim, X_test_interim



In [28]:
# Data preparation
train, test, y_test = prepare_data('FD004.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50','P30','Nf','Nc','Ps30','phi',
                'NRf','NRc','BPR','htBleed','W31','W32'] # selection based on main_notebook

drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

(61249, 27) (41214, 26) (248, 1)


In [29]:
# Model fit
# # Modeling

if True:
    
#     print(elm)
    print('...')
    sequence_length = 40
    alpha = 0.5
    upper = 100
    learning_rate_ = 0.001
    dropout = 0.2
    activation = 'tanh'
    epochs = 20
    batch_size = 128
    train = rul_piecewise_fct(train, upper)
    weights_file = str(alpha) + '1lstm_hyper_parameter_weights.h5'
    input_shape = (sequence_length, len(remaining_sensors))
    model = create_lstm_1layer(dropout, activation, weights_file, input_shape=input_shape)
    mse = []

    # Data preparation
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    print(train_array.shape, label_array.shape, test_array.shape)

    # create train-val split
    # X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)
    
    gss = GroupShuffleSplit(n_splits=3, train_size=0.80, random_state=42)
    
    mse_val = []
    R2_val = []
    RMSE = []
    score_val = []
    test_rul = rul_piecewise_fct(y_test,upper)
    
    with tf.device('/device:GPU:0'):
        for train_unit, val_unit in gss.split(X_train_interim['Unit'].unique(), groups=X_train_interim['Unit'].unique()):
            train_unit = X_train_interim['Unit'].unique()[train_unit]  # gss returns indexes and index starts at 1
            train_split_array = gen_data_wrapper(X_train_interim, sequence_length, remaining_sensors, train_unit)
            train_split_label = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'], train_unit)

            val_unit = X_train_interim['Unit'].unique()[val_unit]
            val_split_array = gen_data_wrapper(X_train_interim, sequence_length, remaining_sensors, val_unit)
            val_split_label = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'], val_unit)

            # train and evaluate model
            model.compile(loss='mse', optimizer='adam')
#             model.load_weights(weights_file)  # reset optimizer and node weights before every training iteration

#             with tf.device('/device:GPU:0'):
            history = model.fit(train_split_array, train_split_label,
                                validation_data=(val_split_array, val_split_label),
                                epochs=epochs,
                                batch_size=batch_size,
                                verbose=0)
#             mse.append(history.history['val_loss'][-1])
            test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
                            for unit_nr in X_test_interim['Unit'].unique())
            test_array = np.concatenate(list(test_gen)).astype(np.float32)
            mse_val.append(history.history['val_loss'][-1])
            y_hat_val_split = model.predict(test_array)
            R2_val.append(r2_score(test_rul, y_hat_val_split))
            RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
            score_val.append(compute_s_score(test_rul, y_hat_val_split))
    d = {'RMSE_val':np.sqrt(mse_val),'RMSE_test': RMSE,'R2_test':np.mean(R2_val), 'Score_test':np.mean(score_val),
                'alpha':alpha, 'rul_piecewise':upper, 'sequence_length':sequence_length}

...


c:\Users\mlndao\OneDrive - Cesi\LamineLCNAM\theseFolder\XAI_comparaison\special_issue\methods.py:94: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.36363636 0.27272727 0.18181818 ... 0.72727273 0.63636364 0.63636364]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_train.loc[df_train['op_cond']==condition, sensor_names] = scaler.transform(df_train.loc[df_train['op_cond']==condition, sensor_names])
c:\Users\mlndao\OneDrive - Cesi\LamineLCNAM\theseFolder\XAI_comparaison\special_issue\methods.py:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.18181818 0.18181818 0.18181818 ... 0.54545455 0.54545455 0.45454545]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_test.loc[df_test['op_cond']==condition, sensor_names] = scaler.transform(df_test.loc[df_test['op_

(51538, 40, 14) (51538, 1) (248, 40, 14)
8/8 [==============================] - 0s 14ms/step


In [30]:
train_array_1c = train_array
test_array_1c = test_array
label_array_1c = label_array

print(" Model evaluation for FD004 : ")
y_hat_train = model.predict(train_array)
evaluate(label_array, y_hat_train, 'train')

y_hat_test = model.predict(test_array)
evaluate(y_test, y_hat_test)
print(' ')
print(" Model evaluation for FD004 : ")

y_hat_test = model.predict(test_array)
evaluate(y_test['RUL'].clip(upper=upper), y_hat_test)
print(' ')
print("S-score for test : ",compute_s_score(y_test, y_hat_test))
print(' ')

 Model evaluation for FD004 : 
1611/1611 [==============================] - 25s 15ms/step
train set RMSE:9.378279685974121, R2:0.9153906790340887
8/8 [==============================] - 0s 12ms/step
test set RMSE:15.87215797809055, R2:0.7912941308175416
 
 Model evaluation for FD004 : 
8/8 [==============================] - 0s 12ms/step
test set RMSE:15.87215797809055, R2:0.7912941308175416
 
S-score for test :  1635.4689368009276
 


In [8]:
from tqdm import tqdm
df_metrics5 = pd.DataFrame()
# model = model_2c
for RD in tqdm(range(5)):
    # Echantillonage
    n_individus = test_array_1c.shape[0]

    # # Choisir aléatoirement 10 indices d'individus
    np.random.seed(RD)
    indices_choisis = np.random.choice(n_individus, size=10, replace=False, )

    # # Sélectionner les données correspondant aux indices choisis
    test_array_sampling = test_array_1c[indices_choisis, :, :]
    label_array_sampling = y_test.values[indices_choisis, :]

    # # Afficher les dimensions des données sélectionnées
    print(test_array_sampling.shape, label_array_sampling.shape)
    # distance matrix XX'
    X_dist = pd.DataFrame(squareform(pdist(test_array_sampling.reshape((test_array_sampling.shape[0], -1)))))

    if True :
        # LIME
        lime_explainer = RecurrentTabularExplainer(test_array, training_labels=label_array,
                                                           feature_names=remaining_sensors,
                                                           mode = 'regression',
                                                           )
        lime_values = get_lime_explanation(test_array_sampling, e = model.predict)
        Lime_dist = pd.DataFrame(squareform(pdist(lime_values))) # Lime values explanation matrix

        #Lime's metrics
        list_metrics_lime = {}
        list_metrics_lime['identity'] = identity(X_dist, Lime_dist)
        list_metrics_lime['separability'] = separability(X_dist, Lime_dist)
        list_metrics_lime['stability'] = stability(X_dist, Lime_dist)
        list_metrics_lime['coherence'], list_metrics_lime['completness'], list_metrics_lime['congruence'] = coherence(model=model.predict, 
                                                    explainer = get_lime_explanation,
                                                   samples=test_array_sampling,
                                                    targets=label_array, e = model.predict)
        list_metrics_lime['selectivity'] = selectivity(model=model.predict, explainer = get_lime_explanation,
                                                   samples=test_array_sampling, e_x = model.predict)
        list_metrics_lime['accumen'] = acumen(get_lime_explanation, test_array_sampling, e=model.predict)
        list_metrics_lime['Verm_stability'] = stability_Velmurugan(get_lime_explanation, test_array_sampling,
                                                                   e=model.predict, top_features=200)
        list_metrics_lime['fidelity'], list_metrics_lime['sparsity'] = fidelity(model=model.predict, 
                                                    explainer = get_lime_explanation,
                                                    samples=test_array_sampling,
                                                    e = model.predict, L2X=True)
        list_metrics_lime['instability'] = instability(model=model.predict, 
                                                    explainer = get_lime_explanation,
                                                    samples=test_array_sampling,
                                                    e = model.predict, L2X=True)
        list_metrics_lime['explainer'] = 'lime_4c'
        df_metrics5 = pd.concat([df_metrics5, pd.DataFrame([list_metrics_lime])])
        print("Lime OK!!")

        # SHAP
        e = KernelSHAP(model)
        shapvalues = get_explainations(test_array_sampling, e)
        shapvalues.shape

        list_metrics_shap = {}
        shap_dist = pd.DataFrame(squareform(pdist(shapvalues))) # shap values explanation matrix

        list_metrics_shap['identity'] = identity(X_dist, shap_dist)
        list_metrics_shap['separability'] = separability(X_dist, shap_dist)
        list_metrics_shap['stability'] = stability(X_dist, shap_dist)
        list_metrics_shap['coherence'], list_metrics_shap['completness'], list_metrics_shap['congruence'] = coherence(model=model.predict, 
                                                    explainer = get_explainations,
                                                   samples=test_array_sampling,
                                                    targets=label_array, e = e)
        list_metrics_shap['selectivity'] = selectivity(model=model.predict, explainer = get_explainations,
                                       samples=test_array_sampling, e_x=e)
        list_metrics_shap['accumen'] = acumen(get_explainations, test_array_sampling, e=e)
        list_metrics_shap['Verm_stability'] = stability_Velmurugan(get_explainations, test_array_sampling,
                                                                   e=e, top_features=200)
        list_metrics_shap['fidelity'], list_metrics_shap['sparsity']= fidelity(model=model.predict, 
                                                    explainer = get_explainations,
                                                    samples=test_array_sampling,
                                                    e = e)
        list_metrics_shap['instability']= instability(model=model.predict, 
                                                    explainer = get_explainations,
                                                    samples=test_array_sampling,
                                                    e = e)

        list_metrics_shap['explainer'] = 'shap_4c'
        df_metrics5 = pd.concat([df_metrics5, pd.DataFrame([list_metrics_shap])])
        print("shap OK !!")

        # L2X
        e = L2X(model.predict, test_array_sampling)
        l2xvalues = get_explainations(test_array_sampling, e, L2X=True)
        l2xvalues.shape

        # l2x's metrics
        list_metrics_l2x = {}
        l2x_dist = pd.DataFrame(squareform(pdist(l2xvalues))) # Lime values explanation matrix

        list_metrics_l2x['identity'] = identity(X_dist, l2x_dist)
        list_metrics_l2x['separability'] = separability(X_dist, l2x_dist)
        list_metrics_l2x['stability'] = stability(X_dist, l2x_dist)
        list_metrics_l2x['coherence'], list_metrics_l2x['completness'], list_metrics_l2x['congruence'] = coherence(model=model.predict, explainer = get_explainations,
                                                   samples=test_array_sampling, targets=label_array_sampling, e = e, L2X=True)
        list_metrics_l2x['selectivity'] = selectivity(model=model.predict, explainer = get_explainations,
                                       samples=test_array_sampling, e_x=e, L2X=True)
        list_metrics_l2x['accumen'] = acumen(get_explainations, test_array_sampling, e=e, L2X=True)
        list_metrics_l2x['Verm_stability'] = stability_Velmurugan(get_explainations, test_array_sampling,
                                                                   e=e, top_features=200, L2X=True)
        list_metrics_l2x['fidelity'], list_metrics_l2x['sparsity']= fidelity(model=model.predict, 
                                                    explainer = get_explainations,
                                                    samples=test_array_sampling,
                                                    e = e, L2X=True)
        list_metrics_l2x['instability'] = instability(model=model.predict, 
                                                    explainer = get_explainations,
                                                    samples=test_array_sampling,
                                                    e = e, L2X=True)
        list_metrics_l2x['explainer'] = 'l2x_4c'
        print("L2X OK !!")

        df_metrics5 = pd.concat([df_metrics5, pd.DataFrame([list_metrics_l2x])])
df_metrics5

  0%|          | 0/5 [00:00<?, ?it/s]

(10, 40, 14) (10, 1)
157/157 [==============================] - 1s 5ms/step
10
157/157 [==============================] - 1s 5ms/step
10
157/157 [==============================] - 1s 5ms/step
10
157/157 [==============================] - 1s 5ms/step
10
157/157 [==============================] - 1s 5ms/step
10
157/157 [==============================] - 1s 5ms/step
10
157/157 [==============================] - 1s 5ms/step
10
157/157 [==============================] - 1s 5ms/step
10
157/157 [==============================] - 1s 5ms/step
10
157/157 [==============================] - 1s 5ms/step
10
157/157 [==============================] - 1s 5ms/step
10
157/157 [==============================] - 1s 5ms/step
10
157/157 [==============================] - 1s 5ms/step
10
157/157 [==============================] - 1s 5ms/step
10
157/157 [==============================] - 1s 5ms/step
10
157/157 [==============================] - 1s 5ms/step
10
157/157 [==============================] - 1s 5ms/s

 20%|██        | 1/5 [1:33:57<6:15:49, 5637.33s/it]

L2X OK !!
(10, 40, 14) (10, 1)
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] - 1s 6ms/step
10
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] - 1s 6ms/step
10
157/157 [==============================] - 1s 6ms/step
10
157/157 [==============================] - 1s 6ms/step
10
157/157 [==============================] - 1s 6ms/step
10
157/157 [==============================] - 1s 5ms/step
10
157/157 [==============================] - 1s 6ms/step
10
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] - 1s 6ms/step
10
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] 

 40%|████      | 2/5 [5:22:58<8:40:13, 10404.41s/it]

...

 80%|████████  | 4/5 [8:55:38<2:09:55, 7795.46s/it]

L2X OK !!
(10, 40, 14) (10, 1)
157/157 [==============================] - 2s 13ms/step
10
157/157 [==============================] - 2s 13ms/step
10
157/157 [==============================] - 2s 13ms/step
10
157/157 [==============================] - 2s 13ms/step
10
157/157 [==============================] - 2s 13ms/step
10
157/157 [==============================] - 2s 13ms/step
10
157/157 [==============================] - 2s 14ms/step
10
157/157 [==============================] - 2s 13ms/step
10
157/157 [==============================] - 2s 13ms/step
10
157/157 [==============================] - 2s 13ms/step
10
157/157 [==============================] - 2s 13ms/step
10
157/157 [==============================] - 2s 13ms/step
10
157/157 [==============================] - 2s 13ms/step
10
157/157 [==============================] - 2s 13ms/step
10
157/157 [==============================] - 2s 14ms/step
10
157/157 [==============================] - 2s 15ms/step
10
157/157 [================

100%|██████████| 5/5 [10:56:07<00:00, 7873.57s/it] 

L2X OK !!


,identity,separability,stability,coherence,completness,congruence,selectivity,accumen,Verm_stability,fidelity,sparsity,instability,explainer
0,1.0,1.0,1.0,0.217896,11.636265,0.223570,0.786064,0.229767,0.287046,0.129994,0.017857,0.475100,lime_4c
0,1.0,1.0,0.9,0.217938,7.242727,0.223821,0.661424,0.217227,1.000000,0.329100,0.017857,0.184998,shap_4c
0,1.0,1.0,0.9,0.253857,36.132486,0.251464,0.717833,0.026941,1.000000,0.249257,0.017857,0.654098,l2x_4c
0,1.0,1.0,1.0,0.174468,39.386192,0.203364,0.677662,0.321825,0.244283,0.117126,0.017857,0.321522,lime_4c
0,1.0,1.0,1.0,0.174426,29.744547,0.203695,0.636673,0.265832,1.000000,0.420433,0.017857,0.298189,shap_4c
0,1.0,1.0,1.0,0.189082,0.236194,0.213804,0.717501,0.045868,1.000000,0.279215,0.017857,0.556757,l2x_4c
0,1.0,1.0,1.0,0.160264,144.057526,0.262086,0.762956,0.285188,0.204461,0.132761,0.017857,0.313037,lime_4c
0,1.0,1.0,0.8,0.160349,199.906860,0.262224,0.697370,0.208408,1.000000,0.203980,0.017857,0.430698,shap_4c
0,1.0,1.0,1.0,0.215299,0.133574,0.304455,0.721060,0.031485,1.000000,0.195118,0.017857,0.393334,l2x_4c
0,1.0,1.0,1.0,0.254614,0.461672,0.275576,0.734405,0.268157,0.217584,0.146892,0.017857,0.380116,lime_4c


In [ ]:
from tqdm import tqdm
sequence_lengths = [30, 35, 38, 39, 40, 50]
list_alpha = [0.2, 0.3, 0.5, 0.6, 0.7]
list_upper = [120,125,130]
metrics_names = ['identity', 'separability', 'stability', 'coherence',
                     'completness', 'congruence', 'selectivity', 'accumen']
df_metrics = pd.DataFrame()
# all_list = [(seq, alpha, upper) for seq in sequence_lengths for alpha in list_alpha for rec_rul in list_upper]
all_list = []
for sq in sequence_lengths:
    for alpha in list_alpha:
        for upper in list_upper:
            triplet = (sq, alpha, upper)
            all_list.append(triplet)
list_d = []
list_df_metric = []

for elm in tqdm(all_list):
    # Modeling
    print(elm)
    print('...')
    sequence_length = elm[0]
    alpha = elm[1]
    upper = elm[2]
    learning_rate_ = 0.001
    dropout = 0.2
    activation = 'tanh'
    epochs = 20
    batch_size = 128
    train = rul_piecewise_fct(train, upper)

    #Model creation
    weights_file = str(alpha) + '1lstm_hyper_parameter_weights.h5'
    input_shape = (sequence_length, len(remaining_sensors))
    model = create_lstm_1layer(dropout, activation, weights_file, input_shape=input_shape)
    mse = []

    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    print(train_array.shape, label_array.shape, test_array.shape)

    # create train-val split
    gss = GroupShuffleSplit(n_splits=3, train_size=0.80, random_state=42)
    
    mse_val = []
    R2_val = []
    RMSE = []
    score_val = []
    test_rul = rul_piecewise_fct(y_test,upper)
    
    with tf.device('/device:GPU:0'):
        for train_unit, val_unit in gss.split(X_train_interim['Unit'].unique(), groups=X_train_interim['Unit'].unique()):
            train_unit = X_train_interim['Unit'].unique()[train_unit]  # gss returns indexes and index starts at 1
            train_split_array = gen_data_wrapper(X_train_interim, sequence_length, remaining_sensors, train_unit)
            train_split_label = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'], train_unit)

            val_unit = X_train_interim['Unit'].unique()[val_unit]
            val_split_array = gen_data_wrapper(X_train_interim, sequence_length, remaining_sensors, val_unit)
            val_split_label = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'], val_unit)

            # train and evaluate model
            model.compile(loss='mse', optimizer='adam')
#             model.load_weights(weights_file)  # reset optimizer and node weights before every training iteration

            # with tf.device('/device:GPU:0'):
            history = model.fit(train_split_array, train_split_label,
                                validation_data=(val_split_array, val_split_label),
                                epochs=epochs,
                                batch_size=batch_size,
                                verbose=0)
#             mse.append(history.history['val_loss'][-1])
            test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
                            for unit_nr in X_test_interim['Unit'].unique())
            test_array = np.concatenate(list(test_gen)).astype(np.float32)
            mse_val.append(history.history['val_loss'][-1])
            y_hat_val_split = model.predict(test_array)
            R2_val.append(r2_score(test_rul, y_hat_val_split))
            RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
            score_val.append(compute_s_score(test_rul, y_hat_val_split))
    d = {'RMSE_val':np.sqrt(mse_val),'RMSE_test': RMSE,'R2_test':np.mean(R2_val), 'Score_test':np.mean(score_val),
                'alpha':alpha, 'rul_piecewise':upper, 'sequence_length':sequence_length}
    list_d.append(d)
    # Get explanation and calculate the score
    # Echantillonage
    n_individus = test_array.shape[0]

    # # Choisir aléatoirement 10 indices d'individus
    np.random.seed(0)
    indices_choisis = np.random.choice(n_individus, size=10, replace=False)

    # Sélectionner les données correspondant aux indices choisis
    test_array_sampling = test_array[indices_choisis, :, :]
    label_array_sampling = y_test.values[indices_choisis, :]

    # Afficher les dimensions des données sélectionnées
    print(test_array_sampling.shape, label_array_sampling.shape)
    
    # distance matrix XX'
    X_dist = pd.DataFrame(squareform(pdist(test_array_sampling.reshape((test_array_sampling.shape[0], -1)))))

    # LIME
    lime_explainer = RecurrentTabularExplainer(test_array, training_labels=label_array,
                                                       feature_names=remaining_sensors,
                                                       mode = 'regression',
                                                       )
    lime_values = get_lime_explanation(test_array_sampling, e = model.predict)
    lime_values.shape
    Lime_dist = pd.DataFrame(squareform(pdist(lime_values))) # Lime values explanation matrix

    #Lime's metrics
    list_metrics_lime = {}
    list_metrics_lime['identity'] = identity(X_dist, Lime_dist)
    list_metrics_lime['separability'] = separability(X_dist, Lime_dist)
    list_metrics_lime['stability'] = stability(X_dist, Lime_dist)
    list_metrics_lime['coherence'], list_metrics_lime['completness'], list_metrics_lime['congruence'] = coherence(model=model.predict, 
                                                    explainer = get_lime_explanation,
                                                   samples=test_array_sampling,
                                                    targets=label_array, e = model.predict)
    list_metrics_lime['selectivity'] = selectivity(model=model.predict, explainer = get_lime_explanation,
                                                   samples=test_array_sampling, e_x = model.predict)
    list_metrics_lime['accumen'] = acumen(get_lime_explanation, test_array_sampling, e=model.predict)
    list_metrics_lime['Verm_stability'] = stability_Velmurugan(get_lime_explanation, test_array_sampling,
                                                                   e=model.predict, top_features=200)
    list_metrics_lime['fidelity'], list_metrics_lime['sparsity'] = fidelity(model=model.predict, 
                                                    explainer = get_lime_explanation,
                                                    samples=test_array_sampling,
                                                    e = model.predict, L2X=True)
    list_metrics_lime['instability'] = instability(model=model.predict, 
                                                    explainer = get_lime_explanation,
                                                    samples=test_array_sampling,
                                                    e = model.predict, L2X=True)
    
    list_metrics_lime['alpha'] = alpha
    list_metrics_lime['upper'] = upper
    list_metrics_lime['sequence_length'] = sequence_length
    list_metrics_lime['explainer'] = 'lime'


    
    
    df_metrics = pd.concat([df_metrics, pd.DataFrame([list_metrics_lime])])
    
    # SHAP
    e = KernelSHAP(model)
    shapvalues = get_explainations(test_array_sampling, e)
    shapvalues.shape

    list_metrics_shap = {}
    shap_dist = pd.DataFrame(squareform(pdist(shapvalues))) # shap values explanation matrix

    list_metrics_shap['identity'] = identity(X_dist, shap_dist)
    list_metrics_shap['separability'] = separability(X_dist, shap_dist)
    list_metrics_shap['stability'] = stability(X_dist, shap_dist)
    list_metrics_shap['coherence'], list_metrics_shap['completness'], list_metrics_shap['congruence'] = coherence(model=model.predict, 
                                                    explainer = get_explainations,
                                                   samples=test_array_sampling,
                                                    targets=label_array, e = e)
    list_metrics_shap['selectivity'] = selectivity(model=model.predict, explainer = get_explainations,
                                       samples=test_array_sampling, e_x=e)
    list_metrics_shap['accumen'] = acumen(get_explainations, test_array_sampling, e=e)
    list_metrics_shap['Verm_stability'] = stability_Velmurugan(get_explainations, test_array_sampling,
                                                                   e=e, top_features=200)
    list_metrics_shap['fidelity'], list_metrics_shap['sparsity']= fidelity(model=model.predict, 
                                                    explainer = get_explainations,
                                                    samples=test_array_sampling,
                                                    e = e)
    list_metrics_shap['instability']= instability(model=model.predict, 
                                                    explainer = get_explainations,
                                                    samples=test_array_sampling,
                                                    e = e)
    
    list_metrics_shap['alpha'] = alpha
    list_metrics_shap['upper'] = upper
    list_metrics_shap['sequence_length'] = sequence_length
    list_metrics_shap['explainer'] = 'shap'

    df_metrics = pd.concat([df_metrics, pd.DataFrame([list_metrics_shap])])
    
    # Saliency
    e = Saliency(model)
    saliencyvalues = get_explainations(test_array_sampling, e)
    saliencyvalues.shape

    # #Saliency's metrics
    list_metrics_saliency = {}
    saliency_dist = pd.DataFrame(squareform(pdist(saliencyvalues))) # Lime values explanation matrix

    list_metrics_saliency['identity'] = identity(X_dist, saliency_dist)
    list_metrics_saliency['separability'] = separability(X_dist, saliency_dist)
    list_metrics_saliency['stability'] = stability(X_dist, saliency_dist)

    list_metrics_saliency['coherence'], list_metrics_saliency['completness'], list_metrics_saliency['congruence'] = coherence(model=model.predict, 
                                                explainer = get_explainations,
                                               samples=test_array_sampling,
                                                targets=label_array, e = e)
    list_metrics_saliency['selectivity'] = selectivity(model=model.predict, explainer = get_explainations,
                                   samples=test_array_sampling, e_x=e)
    list_metrics_saliency['accumen'] = 'NA'
    
    list_metrics_saliency['alpha'] = alpha
    list_metrics_saliency['upper'] = upper
    list_metrics_saliency['sequence_length'] = sequence_length
    list_metrics_saliency['explainer'] = 'saliency'
    df_metrics = pd.concat([df_metrics, pd.DataFrame([list_metrics_saliency])])
    
    # L2X
    e = L2X(model.predict, test_array_sampling)
    l2xvalues = get_explainations(test_array_sampling, e, L2X=True)
    l2xvalues.shape

    # Saliency's metrics
    list_metrics_l2x = {}
    l2x_dist = pd.DataFrame(squareform(pdist(l2xvalues))) # Lime values explanation matrix

    list_metrics_l2x['identity'] = identity(X_dist, l2x_dist)
    list_metrics_l2x['separability'] = separability(X_dist, l2x_dist)
    list_metrics_l2x['stability'] = stability(X_dist, l2x_dist)
    list_metrics_l2x['coherence'], list_metrics_l2x['completness'], list_metrics_l2x['congruence'] = coherence(model=model.predict, explainer = get_explainations,
                                                   samples=test_array_sampling, targets=label_array_sampling, e = e, L2X=True)
    list_metrics_l2x['selectivity'] = selectivity(model=model.predict, explainer = get_explainations,
                                       samples=test_array_sampling, e_x=e, L2X=True)
    list_metrics_l2x['accumen'] = acumen(get_explainations, test_array_sampling, e=e, L2X=True)
    list_metrics_l2x['Verm_stability'] = stability_Velmurugan(get_explainations, test_array_sampling,
                                                                   e=e, top_features=200, L2X=True)
    list_metrics_l2x['fidelity'], list_metrics_l2x['sparsity']= fidelity(model=model.predict, 
                                                    explainer = get_explainations,
                                                    samples=test_array_sampling,
                                                    e = e, L2X=True)
    list_metrics_l2x['instability'] = instability(model=model.predict, 
                                                    explainer = get_explainations,
                                                    samples=test_array_sampling,
                                                    e = e, L2X=True)
    
    list_metrics_l2x['alpha'] = alpha
    list_metrics_l2x['upper'] = upper
    list_metrics_l2x['sequence_length'] = sequence_length
    list_metrics_l2x['explainer'] = 'l2x'

    
    df_metrics = pd.concat([df_metrics, pd.DataFrame([list_metrics_l2x])])